# Creating a DataFrame with the neighborhoods of Toronto

Notebook Created by Rafael Serrano to complete the requirements of the IBM Data Science Capstone in Coursera

In [2]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
!conda install -c conda-forge beautifulSoup4 --yes

Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda

  added / updated specs: 
    - beautifulsoup4


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    openssl-1.0.2p             |       h470a237_1         3.1 MB  conda-forge
    certifi-2018.10.15         |        py36_1000         138 KB  conda-forge
    ca-certificates-2018.10.15 |       ha4d7672_0         135 KB  conda-forge
    beautifulsoup4-4.6.3       |        py36_1000         141 KB  conda-forge
    conda-4.5.11               |        py36_1000         651 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         4.2 MB

The following packages will be UPDATED:

    beautifulsoup4:  4.6.0-py36h49b8c8c_1             --> 4.6.3-py36_1000       conda-forge
    ca-certificates: 2018.8.24-ha4d7672_0 conda-forge --> 

In [72]:
from bs4 import BeautifulSoup
import requests

In [174]:
r = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
soup = BeautifulSoup(r.content, 'lxml')
table = soup.find_all('table')[0]
df = pd.read_html(str(table), header=0)

df = pd.DataFrame(df[0])
df.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


## Remove the postal codes not assigned to any Borough

In [173]:
df = df[df.Borough != 'Not assigned']
df.head()

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights


## Change the values where the Neighborhood is not assigned to be the same as the Borough

In [167]:
df['Neighbourhood'] = np.where(df.Neighbourhood.values == 'Not assigned', df.Borough.values, df.Neighbourhood.values)

/home/jupyterlab/conda/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [168]:
#df.Neighbourhood[df.Neighbourhood == 'Not assigned'] = "Queen's Park"
#df.loc[8,"Neighbourhood"]= "Queen's Park"
#df.loc[8]

In [169]:
df[df.Neighbourhood == 'Not assigned']

,Postcode,Borough,Neighbourhood


## Group the Boroughs and join the Neighborhoods in one column 

In [170]:
df_to = df.groupby(['Postcode','Borough'])['Neighbourhood'].apply(lambda x: "%s" % ', '.join(x)).reset_index()
df_to = pd.DataFrame(df_to)
df_to.head()

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


Rename the columns to resemble the example posted in the assignment

In [171]:
df_to.columns = ['PostalCode', 'Borough', 'Neighborhood']
df_to.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


Finally we display the shape of the dataframe

In [172]:
df_to.shape

(103, 3)